<a href="https://colab.research.google.com/github/DanqingLiu/cs636/blob/master/Titanic_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !cp /content/kaggle.json /root/.kaggle/
# !chmod 600 /root/.kaggle/kaggle.json
# !kaggle competitions download -c titanic
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Data Preprocessing

### Discard ' Name', 'PassengerId', 'Ticket', 'Cabin'. 
### Drop samples with missing data.
### Fill missing 'Age' and 'Fare' with the median value in test data.

In [0]:
train_data = pd.read_csv("/content/train.csv")
test_data = pd.read_csv("/content/test.csv")
# print(train_data.shape) #(891, 12)
# print(test_data.shape) #(418, 11)

#print(train_data.columns.values) #['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch' 'Ticket' 'Fare' 'Cabin' 'Embarked']
# train_data.head()
# train_data.info()
# train_data.describe()
# train_data.describe(include=['O'])
# train_data[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)
# g = sns.FacetGrid(train_data, col='Survived', row='Pclass')
# g.map(plt.hist, 'Age', bins=20)
# g.add_legend()

In [0]:
train_data = train_data.drop(['Name', 'PassengerId', 'Ticket', 'Cabin'], axis=1)
X_test = test_data.drop(['Name', 'PassengerId', 'Ticket', 'Cabin'], axis=1)
train_data = train_data.dropna()

train_data['Sex'] = train_data['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
X_test['Sex'] = X_test['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
train_data['Embarked'] = train_data['Embarked'].map( {'S': 2, 'C': 1, 'Q': 0} ).astype(int)
X_test['Embarked'] = X_test['Embarked'].map( {'S': 2, 'C': 1, 'Q': 0} ).astype(int)

X_train = train_data.drop(['Survived'], axis=1)
Y_train = train_data['Survived']

X_test.loc[(X_test.Age.isnull()), 'Age'] = X_test['Age'].median()
X_test.loc[(X_test.Fare.isnull()), 'Fare'] = X_test['Fare'].median()


## Build NN model

In [93]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu',input_shape=(7,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['accuracy'])
model.fit(X_train,Y_train,epochs=10,batch_size=16)


Epoch 1/10
712/712 [==============================] - 1s 715us/step - loss: 0.8219 - acc: 0.6826
Epoch 2/10
712/712 [==============================] - 0s 72us/step - loss: 0.6696 - acc: 0.6826
Epoch 3/10
712/712 [==============================] - 0s 70us/step - loss: 0.6462 - acc: 0.6882
Epoch 4/10
712/712 [==============================] - 0s 70us/step - loss: 0.6378 - acc: 0.7051
Epoch 5/10
712/712 [==============================] - 0s 68us/step - loss: 0.6178 - acc: 0.7135
Epoch 6/10
712/712 [==============================] - 0s 72us/step - loss: 0.6358 - acc: 0.6994
Epoch 7/10
712/712 [==============================] - 0s 69us/step - loss: 0.6198 - acc: 0.7065
Epoch 8/10
712/712 [==============================] - 0s 69us/step - loss: 0.6081 - acc: 0.7163
Epoch 9/10
712/712 [==============================] - 0s 72us/step - loss: 0.6184 - acc: 0.7177
Epoch 10/10
712/712 [==============================] - 0s 70us/step - loss: 0.6066 - acc: 0.7261


## Predict with the model (Test accuracy: 66.9%)

In [0]:
y_pred = model.predict(X_test)
label = (y_pred > 0.5)
PassengerId = (test_data['PassengerId'])
PassengerId = PassengerId.values.reshape(418,1)

np.savetxt("submit.csv", np.concatenate((PassengerId, label), axis=1), "%d,%d", header="PassengerId,Survived", comments='')

## LogisticRegression (Test accuracy: 73.7%)

In [0]:
from sklearn.linear_model import LogisticRegression

In [105]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
# acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
# acc_log
Y_pred = Y_pred.reshape(418,1)
PassengerId = (test_data['PassengerId'])
PassengerId = PassengerId.values.reshape(418,1)

np.savetxt("submit-logistic-regression.csv", np.concatenate((PassengerId, Y_pred), axis=1), "%d,%d", header="PassengerId,Survived", comments='')

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## Decision tree (Test accuracy: 63.6%)

In [0]:
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
# acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
# acc_decision_tree
Y_pred = Y_pred.reshape(418,1)
PassengerId = (test_data['PassengerId'])
PassengerId = PassengerId.values.reshape(418,1)

np.savetxt("submit-decision-tree.csv", np.concatenate((PassengerId, Y_pred), axis=1), "%d,%d", header="PassengerId,Survived", comments='')